# Análisis de comunidades microbianas usando Qiime2-2019

Laura Brenes Guillén

Modificado de 

https://qiime2.org/

# Introducción al uso del cluster

El análisis de puede correr en una computadora personal, sin embargo qiime2 debe instalarse en su computadora. 
Para instalar Qiime2 en su computadora puede seguir el siguiente tutorial: https://docs.qiime2.org/2019.1/install/native/#install-miniconda


Para evitar hacer la instalación de qiime2 en la computadora se puede utilizar el cluster del CNCA. 

Para ingresar el cluster debe abrir la terminar de su computadora y escribir


In [ ]:
ssh username@cluster.cenat.ac.cr

El username corresponde a su usuario personal

Una vez escrito esto en el cluster saldrá la opción de colocar su contraseña

Debe tener sus archivos .fastq.gz en la carpeta en donde quiera hacer su análisis

# SESIÓN INTERACTIVA DEL CLUSTER-MODULO

Debido a que algunos comando consumen mucha memoria RAM es mejor correr todos los análisis desde la sesión interactiva. Al ingresar al cluster usted se encuentra dentro de login, por lo que debe ir a la sesión interactiva. 
Para ello debe utilizar el siguiente comando

In [ ]:
qsub -I -q phi-debug

Cargar modulo en el cluster


In [ ]:
module load miniconda/qiime2-2018.11 

# METADATOS

Crear archivo de metadatados con la información de las muestras.
El formato del archvio debe de ser .tsv, este puede ser creado en google sheets. 
Debe contener el ID de las muestras (ver detalles en archivo adjunto). El archivo se llamará sample metadata.tsv
Una vez se ha generado el archivo este debe modificarse en formato para qiime2 con el siguiente comando.
Debe generarse un archivo llamado "tabulated-metadata.qzv"


Recuerde incluir variables categóricas.. será útil de pasos posteriores

In [ ]:
qiime metadata tabulate
--m-input-file sample-metadata.tsv 
--o-visualization tabulated-metadata.qzv 


Para la visualización interactiva de la tabla debe cargar el archivo en la siguiente página https://view.qiime2.org/

Para importar los archivos .fastq.gz en formato Qiime2 debe generar un archivo manifiesto, en el cual se indica el nombre de la muestra, el path y la dirección de las secuencias, ya sea forward o reverse. 
Una vez se ha creado el archivo manifiesto (adjunto), debe correrse el siguiente comando

Para importar otro tipo de archivo puede consultar el siguiente link (desmultiplexed, pair-end)

https://docs.qiime2.org/2018.11/tutorials/importing/

In [ ]:
qiime tools import --type 'SampleData[SequencesWithQuality]' 
--input-path se-33-manifest.txt 
--output-path single-end-sequences.qza 
--input-format SingleEndFastqManifestPhred33 

Este comando funciona cuando se tienen archivo desmultiplexados y single end, para archivo multiplexados o paired end puede consultar el manual
https://docs.qiime2.org/2017.12/tutorials/importing/?highlight=import

si tiene instalado el qiime2 en su compu debe modificar el flag --source-format 

Para revisar si el archivo .qza es compatible con qiime2-view puede correrse el siguiente comando


In [ ]:
qiime tools peek single-end-sequences.qza

# ANÁLISIS DE CALIDAD DE LAS SECUENCIAS

Este análisis permite determinar cuántas secuencias se obtuvieron por muestra y también obtener un resumen de la distribución de las calidades de las secuencias en cada posición

El archivo demuz.qzv puede ser visualizado en la interfaz interactiva de qiime2

In [ ]:
qiime demux summarize 
--i-data single-end-sequences.qza 
--o-visualization demux.qzv 

Para eliminar las secuencias de "baja calidad" se puede utilizar el siguiente comando. El valor 220 es seleccionado debido a que en esa posición en donde se observa un Phredscore de 20. Este valor es decidido por el usuario

NOTA: Cuando se corre en el cluster no es tan rápido como correrlo en la servidor del curso, tarda mucho más. Hacer un archivo PBS (En 6 muestras single end (sol forward) tardó casi 72 horas. 


In [ ]:
qiime dada2 denoise-single 
  --i-demultiplexed-seqs single-end-demux.qza 
  --p-trim-left 0 
  --p-trunc-len 220 
  --o-representative-sequences rep-seqs.qza 
  --o-table table.qza 
  --o-denoising-stats stats-dada2.qza

Visualización de los datos después de eliminar secuencias de baja calidad

In [ ]:
qiime metadata tabulate 
  --m-input-file stats-dada2.qza 
  --o-visualization stats-dada2.qzv

# ANÁLISIS FILOGENÉTICO

In [ ]:
qiime phylogeny align-to-tree-mafft-fasttree 
  --i-sequences rep-seqs.qza 
  --o-alignment aligned-rep-seqs.qza 
  --o-masked-alignment masked-aligned-rep-seqs.qza 
  --o-tree unrooted-tree.qza 
  --o-rooted-tree rooted-tree.qza

Visualización de los datos 

In [ ]:
qiime feature-table summarize 
  --i-table table.qza 
  --o-visualization table.qzv 
  --m-sample-metadata-file sample-metadata.tsv

qiime feature-table tabulate-seqs 
  --i-data rep-seqs.qza 
  --o-visualization rep-seqs.qzv

# Alfa y Beta Diversidad (recordar modificar el sampling dept según sus datos)

In [ ]:
qiime diversity core-metrics-phylogenetic 
  --i-phylogeny rooted-tree.qza 
  --i-table table.qza 
  --p-sampling-depth 1109 
  --m-metadata-file sample-metadata.tsv 
  --output-dir core-metrics-results

Diversidad alfa (variables categóricas)

In [ ]:
qiime diversity alpha-group-significance 
  --i-alpha-diversity core-metrics-results/faith_pd_vector.qza 
  --m-metadata-file sample-metadata.tsv 
  --o-visualization core-metrics-results/faith-pd-group-significance.qzv

Diversidad Beta (variables categóricas)

In [ ]:
qiime diversity beta-group-significance 
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza 
  --m-metadata-file sample-metadata.tsv 
  --m-metadata-column BodySite 
  --o-visualization core-metrics-results/unweighted-unifrac-body-site-significance.qzv 
  --p-pairwise

# Análisis de Componentes Principales

In [ ]:
qiime emperor plot 
  --i-pcoa core-metrics-results/unweighted_unifrac_pcoa_results.qza 
  --m-metadata-file sample-metadata.tsv 
  --p-custom-axes Profundidad 
  --o-visualization core-metrics-results/unweighted-unifrac-emperor-Profundidad.qzv

# Identificación y remoción de quimeras

Identificación de quimeras de novo (https://docs.qiime2.org/2018.11/tutorials/chimera/)

In [ ]:
qiime vsearch uchime-denovo 
  --i-table table.qza 
  --i-sequences rep-seqs.qza 
  --output-dir uchime-dn-out

In [ ]:
Al correr este comando se va a generar una carpeta llama uchime-dn-out
Esta carpeta contiene los archivos que se utilizarán en pasos posteriores

In [ ]:
Estadísticas de la identificación de quimeras
Para mayor información de las estadísticas generadas puede consultarse el siguiente link https://docs.qiime2.org/2018.11/plugins/available/vsearch/uchime-ref/

In [ ]:
qiime metadata tabulate 
  --m-input-file uchime-dn-out/stats.qza 
  --o-visualization uchime-dn-out/stats.qzv

Filtrar cuadro de abundancia "otu-table" (remoción de quimeras) 

In [ ]:
qiime feature-table filter-features 
  --i-table table.qza 
  --m-metadata-file uchime-dn-out/nonchimeras.qza 
  --o-filtered-table uchime-dn-out/table-nonchimeric-wo-borderline.qza

In [ ]:
qiime feature-table filter-seqs 
  --i-data rep-seqs.qza 
  --m-metadata-file uchime-dn-out/nonchimeras.qza 
  --o-filtered-data uchime-dn-out/rep-seqs-nonchimeric-wo-borderline.qza

In [ ]:
qiime feature-table summarize 
  --i-table uchime-dn-out/table-nonchimeric-wo-borderline.qza 
  --o-visualization uchime-dn-out/table-nonchimeric-wo-borderline.qzv

In [ ]:
Los archivos table-nonchimeric-wo-borderline.qza y rep-seqs-nonchimeric-wo-borderline.qza serán utilizados para asignar la clasificación taxonómica

# Identificación taxonómica de las secuencias

Para realizar el análisis filogenético deben descargarse la base de datos entrenada de Greengenes o de Silva. 
Puede encontrar las bases en el siguiente link https://docs.qiime2.org/2018.11/data-resources/. 

Si la base no está cortada en la región de interés (V3-v4, por ejemplo) puede utilizarse la base de datos completa. Este paso puede durar algún tiempo, por lo que se recomienda utilizar el archivo PBS (ver adjunto)

Puede utilizar el siguiente comando para descargar la base Silva

In [ ]:
wget -O "silva-132-99-nb-classifier.qza" "https://data.qiime2.org/2018.11/common/silva-132-99-nb-classifier.qza"

In [ ]:
qiime feature-classifier classify-sklearn \
  --i-classifier silva-132-99-nb-classifier.qza \
  --i-reads rep-seqs-nonchimeric-wo-borderline.qza \
  --o-classification taxonomy_nochimeras.qza

In [ ]:
qiime metadata tabulate \
  --m-input-file taxonomy.qza \
  --o-visualization taxonomy.qzv

In [ ]:
qiime taxa barplot \
  --i-table table-nonchimeric-wo-borderline.qza \
  --i-taxonomy taxonomy_nochimeras.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization taxa-bar-plots.qzv

# Exportar tabla del conteo de cada feature (OTU) por muestra

Tabla de conteo de cada secuencia por muestra (sin asignación taxonómica)
Dentro de la carpeta "extracted-feature-table" se encontratrá una carpeta con un "conjunto de números", dentro de esa carpeta encontrará la carpeta "data". 
Dentro de la carpeta "data" se encuentra el archivo .biom

In [ ]:
mkdir extracted-feature-table
qiime tools extract \
  --input-path table-nonchimeric-wo-borderline.qza \
  --output-path extracted-feature-table

Para visualizar el archivo .biom en excell, puede utilizar el siguiente comando

In [ ]:
biom convert -i table.biom -o table.from_biom_w_taxonomy.txt --to-tsv --header-key taxonomy

# Exportar la tabla de taxonomía

In [ ]:
qiime tools export \
  --input-path taxonomy_nochimeras.qza \
  --output-path exported-taxonomy-table

In [ ]:
qiime feature-table filter-features \
  --i-table taxonomy_nochimeras.qza \
  --m-metadata-file uchime-dn-out/nonchimeras.qza \
  --o-filtered-table uchime-dn-out/table-taxonomy-nonchimeric-wo-borderline.qza

# Exportar tablas L1,L2...(similar a qiime1)

Tabla de conteo de cada secuencia por muestra (similar a resultados de taxonomy summary en qiime1, L1,L2,L3,...)

In [ ]:
qiime taxa collapse \
  --i-table table-nonchimeric-wo-borderline.qza \
  --i-taxonomy taxonomy_nochimeras.qza \
  --p-level 6 \
  --output-dir taxtable6/

Se va a generar una carpeta con el nombre de "taxtable" dentro de la cual se encuentra un archivo .qza que debe modificarse para generar un archivo .biom

In [ ]:
mkdir extracted-feature-table
qiime tools extract \
  --input-path collapsed_table.qza \
  --output-path extracted-feature-table

Para visualizar el archivo .biom en excell (dentro de carpeta "extracted-feature-table/1512165sss/data/", puede utilizar el siguiente comando

In [ ]:
biom convert -i table.biom -o table.from_biom_w_taxonomy.txt --to-tsv --header-key taxonomy

# Si sus datos lo requieren usted puede eliminar cloroplastos, mitocondrias y singletons

# Eliminar cloroplastos-mitocondrias

In [ ]:
qiime taxa filter-table \
  --i-table table-nonchimeric-wo-borderline.qza \
  --i-taxonomy taxonomy_nochimeras.qza \
  --p-exclude mitochondria,chloroplast \
  --o-filtered-table table-no-chimeras-no-mitochondria-no-chloroplast.qza

# Eliminar singletons (https://docs.qiime2.org/2018.4/tutorials/filtering/)

In [ ]:
qiime feature-table filter-features \
  --i-table table-nonchimeric-wo-borderline.qza \
  --p-min-frequency 1 \
  --o-filtered-table table-no-chimeras-no-mitochondria-no-chloroplast-no-singles.qza

# Mezclar datos de diferentes análisis

In [ ]:
qiime feature-table merge \
    --i-table1 analysis1/table.qza \
    --i-table2 analysis2/table.qza \
    --o-merged-table merged-table.qza

qiime feature-table merge-taxa-data \
    --i-data1 analysis1/taxonomy.qza \
    --i-data2 analysis2/taxonomy.qza \
    --o-merged-data merged-taxonomy.qza